In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio

In [ ]:
# Generate all (num_classes)^n strings in support of MRF
def generate_strings(n, num_classes):
    assert num_classes >= 2 and num_classes < 10
    if n == 1:
        return [str(i) for i in range(num_classes)]
    ret = []
    all_smaller_strings = generate_strings(n-1, num_classes)
    for i in range(num_classes):
        ret.extend([str(i) + s for s in all_smaller_strings])
    return ret

# Convert vector to string
def get_string_from_vector(v):
    ret = ""
    for digit in v:
        assert digit >=0 and digit < 10
        ret += str(digit)
    return ret

# Get f = \sum_{ij}Aij\delta(i, j)/2 + \sum_i\sum_k b_ik\delta(i, k)
def get_f(A, h, s):
    num_classes = h.shape[1]
    n = A.shape[0]
    if type(s) == str:
        s = np.array(list(s), dtype=int)
    delta = np.zeros((num_classes, n))
    delta[s, np.arange(n)] = 1
    sm = np.sum((delta.T @ delta) * A) - np.sum(A) / 2
    truth = np.eye(num_classes)
    sm += 2 * np.sum((delta.T @ truth) * h) - np.sum(h)
    return sm

# Get ground truth logZ and mode
def ground_truth(A, h, all_strings):
    n = len(A)
    mx = float("-inf")
    num_classes = h.shape[1]
    sm_list = []
    for s in all_strings:
        sm = get_f(A, h, s)    
        mx = max(mx, sm)
        sm_list.append(sm)
    sm_list = [elem - mx for elem in sm_list]
    logZ = np.log(np.sum(np.exp(sm_list))) + mx
    
    return logZ, mx

In [ ]:
# Define MRF parameters
cw_range = [0.1,  0.5,  1.,   1.5,  2.,   2.5,  3.,   3.5,  4.,   4.5,  5.,   5.5,  6.,   6.5,
7.,   7.5,  8.,   8.5,  9.,   9.5, 10.]
cw_len = len(cw_range)
num_samples = 100
num_classes = 4
n = 8
# for num_classes = 3, n = 10
# for num_classes = 4, n = 8
# for num_classes = 5, n = 7

<h2> Multi-class MRF complete graph data generation

In [ ]:
coupling_data = np.zeros((n, n, len(cw_range), num_samples))
bias_data = np.zeros((n, num_classes, len(cw_range), num_samples))
logZ_data = np.zeros((len(cw_range), num_samples))
modes_data = np.zeros((len(cw_range), num_samples))
all_strings = generate_strings(n, num_classes)
for i, cw in enumerate(cw_range):
    for sample in range(num_samples):
        print(i, sample)
        A = np.random.uniform(low=-cw, high=cw, size=(n, n))
        for j in range(n):
            A[j, j] = 0
        A = (A + A.T) / 2
        h = np.random.uniform(low=-1, high=1, size=(n, num_classes))
        logZ, mode = ground_truth(A, h, all_strings)
        coupling_data[:, :, i, sample] = A
        bias_data[:, :, i, sample] = h
        logZ_data[i, sample] = logZ
        modes_data[i, sample] = mode